# Python Code

Here we are importing libraries and creating necessary funcitions,
and following next cell contains main function.


In [2]:
import copy
import datetime
import numpy as np
import pandas as pd
import os
import subprocess
import sys
import math
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from queue import PriorityQueue

from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#from utils.custom_dataset import load_std_regress_data,load_dataset_custom
#from utils.Create_Slices import get_slices
#from model.LinearRegression import RegressionNet, LogisticNet
#from model.SELCON import FindSubset_Vect_No_ValLoss as FindSubset_Vect, FindSubset_Vect_TrnLoss
#from model.facility_location import run_stochastic_Facloc

from torch.utils.data import DataLoader

torch.manual_seed(42)
np.random.seed(42)

class FindSubset_Vect(object):
    def __init__(self, x_trn, y_trn, x_val, y_val,model,loss,device,delta,lr,lam,batch):
        
        self.x_trn = x_trn
        self.y_trn = y_trn
        #self.trn_batch = trn_batch

        self.x_val = x_val
        self.y_val = y_val

        self.model = model
        self.criterion = loss 
        self.device = device

        self.delta = delta
        self.lr = lr
        self.lam = lam
        #self.optimizer = optimizer
        self.batch_size = batch

        torch.manual_seed(42)
        torch.cuda.manual_seed(42)
        np.random.seed(42)


    def precompute(self,f_pi_epoch,p_epoch,alphas):
        '''
        
        '''

        main_optimizer = torch.optim.Adam([
                {'params': self.model.parameters()}], lr=self.lr)
                
        dual_optimizer = torch.optim.Adam([{'params': alphas}], lr=self.lr)

        print("starting Pre compute")
        #alphas = torch.rand_like(self.delta,requires_grad=True) 

        #print(alphas)

        #scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=[10,20,40,100],\
        #    gamma=0.5) #[e*2 for e in change]

        #alphas.requires_grad = False
        loader_val = DataLoader(CustomDataset(self.x_val, self.y_val, device = self.device, transform=None),\
            shuffle=False, batch_size=self.batch_size)
            

        #Compute F_phi
        #for i in range(f_pi_epoch):

        prev_loss = 1000
        stop_count = 0
        i=0
        
        while(True):
            
            main_optimizer.zero_grad()
            
            '''l2_reg = 0
            for param in self.model.parameters():
                l2_reg += torch.norm(param)'''

            #l = [torch.flatten(p) for p in main_model.parameters()]
            #flat = torch.cat(l)
            #l2_reg = torch.sum(flat*flat)
            
            constraint = 0. 
            for batch_idx in list(loader_val.batch_sampler):
                    
                inputs, targets = loader_val.dataset[batch_idx]
                inputs, targets = inputs.to(self.device), targets.to(self.device)
            
                val_out = self.model(inputs)
                constraint += self.criterion(val_out, targets)
                
            constraint /= len(loader_val.batch_sampler)
            constraint = constraint - self.delta
            multiplier = alphas*constraint #torch.dot(alphas,constraint)

            loss = multiplier
            self.F_phi = loss.item()
            loss.backward()
            
            main_optimizer.step()
            #scheduler.step()
            
            '''for param in self.model.parameters():
                param.requires_grad = False
            alphas.requires_grad = True'''

            dual_optimizer.zero_grad()

            constraint = 0.
            for batch_idx in list(loader_val.batch_sampler):
                    
                inputs, targets = loader_val.dataset[batch_idx]
                inputs, targets = inputs.to(self.device), targets.to(self.device)
            
                val_out = self.model(inputs)
                constraint += self.criterion(val_out, targets)
            
            constraint /= len(loader_val.batch_sampler)
            constraint = constraint - self.delta
            multiplier = -1.0*alphas*constraint #torch.dot(-1.0*alphas,constraint)
            
            multiplier.backward()
            
            dual_optimizer.step()

            alphas.requires_grad = False
            alphas.clamp_(min=0.0)
            alphas.requires_grad = True
            #print(alphas)

            '''for param in self.model.parameters():
                param.requires_grad = True'''

            if loss.item() <= 0.:
                break

            #if i>= f_pi_epoch:
            #    break

            if abs(prev_loss - loss.item()) <= 1e-1 and stop_count >= 5:
                break 
            elif abs(prev_loss - loss.item()) <= 1e-1:
                stop_count += 1
            else:
                stop_count = 0

            prev_loss = loss.item()
            i+=1

            #if i % 50 == 0:
            #    print(loss.item(),alphas,constraint)

        print("Finishing F phi")

        if loss.item() <= 0.:
            alphas = torch.zeros_like(alphas)

        print(loss.item())

        l = [torch.flatten(p) for p in self.model.state_dict().values()]
        flat = torch.cat(l).detach().clone()
        
        '''main_optimizer = torch.optim.Adam([{'params': self.model.parameters()}], lr=self.lr)
        
        alphas = torch.ones(alphas.shape) * 1e-4
        x_val_ext = torch.cat((self.x_val,torch.ones(self.x_val.shape[0],device=self.device).view(-1,1))\
                ,dim=1)

        yTy = torch.dot(self.y_val,self.y_val)
        
        #yTX = alphas*torch.matmul(self.y_val.view(1,-1),x_val_ext)

        XTX = torch.inverse(alphas*torch.matmul(x_val_ext.T,x_val_ext))

        XTy = alphas*torch.matmul(x_val_ext.T,self.y_val.view(-1,1))

        accum = torch.matmul(XTX, XTy) #torch.matmul(XTy,XTX)

        flat = alphas*yTy - alphas*self.delta - accum'''
        
        self.F_values = torch.zeros(len(self.x_trn),device=self.device)

        device_new = self.device #"cuda:2"#self.device #
        beta1,beta2 = main_optimizer.param_groups[0]['betas']
        #main_optimizer.param_groups[0]['eps']

        loader_tr = DataLoader(CustomDataset_WithId(self.x_trn, self.y_trn,\
            transform=None),shuffle=False,batch_size=self.batch_size*20)

        loader_val = DataLoader(CustomDataset(self.x_val, self.y_val,device = self.device,transform=None),\
            shuffle=False,batch_size=self.batch_size*20)
        
        for batch_idx in list(loader_tr.batch_sampler):

            inputs, targets, idxs = loader_tr.dataset[batch_idx]
            inputs, targets = inputs.to(self.device), targets.to(self.device)

            ele_delta = self.delta.repeat(targets.shape[0]).to(self.device)
        
            weights = flat.view(1,-1).repeat(targets.shape[0], 1)
            ele_alphas = alphas.detach().repeat(targets.shape[0]).to(self.device)
            #print(weights.shape)

            exp_avg_w = torch.zeros_like(weights)
            exp_avg_sq_w = torch.zeros_like(weights)

            #exp_avg_a = torch.zeros_like(ele_alphas)
            #exp_avg_sq_a = torch.zeros_like(ele_alphas)

            exten_inp = torch.cat((inputs,torch.ones(inputs.shape[0],device=self.device).view(-1,1))\
                ,dim=1)

            bias_correction1 = 1.0 
            bias_correction2 = 1.0 

            for i in range(p_epoch):

                trn_loss_g = torch.sum(exten_inp*weights,dim=1) - targets
                fin_trn_loss_g = exten_inp*2*trn_loss_g[:,None]

                #no_bias = weights.clone()
                #no_bias[-1,:] = torch.zeros(weights.shape[0])
                
                weight_grad = fin_trn_loss_g+ 2*self.lam*\
                    torch.cat((weights[:,:-1], torch.zeros((weights.shape[0],1),device=self.device)),dim=1)
                #+fin_val_loss_g*ele_alphas[:,None]

                exp_avg_w.mul_(beta1).add_(1.0 - beta1, weight_grad)
                exp_avg_sq_w.mul_(beta2).addcmul_(1.0 - beta2, weight_grad, weight_grad)
                denom = exp_avg_sq_w.sqrt().add_(main_optimizer.param_groups[0]['eps'])

                bias_correction1 *= beta1
                bias_correction2 *= beta2
                step_size = (self.lr) * math.sqrt(1.0-bias_correction2) / (1.0-bias_correction1)
                weights.addcdiv_(-step_size, exp_avg_w, denom)
                
            val_losses = 0.
            for batch_idx_val in list(loader_val.batch_sampler):
                    
                inputs_val, targets_val = loader_val.dataset[batch_idx_val]
                inputs_val, targets_val = inputs_val.to(self.device), targets_val.to(self.device)

                exten_val = torch.cat((inputs_val,torch.ones(inputs_val.shape[0],device=self.device).view(-1,1)),dim=1)
                
                exten_val_y = torch.mean(targets_val).repeat(min(self.batch_size*20,targets.shape[0]))

                val_loss = torch.sum(weights*torch.mean(exten_val,dim=0),dim=1) - exten_val_y

                val_losses+= val_loss*val_loss #torch.mean(val_loss*val_loss,dim=0)
            
            reg = torch.sum(weights[:,:-1]*weights[:,:-1],dim=1)
            trn_loss = torch.sum(exten_inp*weights,dim=1) - targets

            self.F_values[idxs] = trn_loss*trn_loss+ self.lam*reg +torch.max(torch.zeros_like(ele_alphas),\
                (val_losses/len(loader_val.batch_sampler)-ele_delta)*ele_alphas)

        print(self.F_values[:10])

        #self.F_values = self.F_values - max(loss.item(),0.) 

        print("Finishing Element wise F")


    def return_subset(self,theta_init,p_epoch,curr_subset,alphas,budget,batch,\
        step,w_exp_avg,w_exp_avg_sq,a_exp_avg,a_exp_avg_sq):
        
        m_values = self.F_values.detach().clone() #torch.zeros(len(self.x_trn))
        
        self.model.load_state_dict(theta_init)

        loader_tr = DataLoader(CustomDataset_WithId(self.x_trn[curr_subset], self.y_trn[curr_subset],\
            transform=None),shuffle=False,batch_size=batch)

        loader_val = DataLoader(CustomDataset(self.x_val, self.y_val,device = self.device,transform=None),\
            shuffle=False,batch_size=batch)  

        sum_error = torch.nn.MSELoss(reduction='sum')       

        with torch.no_grad():

            F_curr = 0.

            for batch_idx in list(loader_tr.batch_sampler):
            
                inputs, targets, _ = loader_tr.dataset[batch_idx]
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                scores = self.model(inputs)
                #print(self.criterion(scores, targets).item())

                F_curr += sum_error(scores, targets).item() 

            #F_curr /= len(loader_tr.batch_sampler)
            #print(F_curr,end=",")

            l = [torch.flatten(p) for p in self.model.parameters()]
            flatt = torch.cat(l)
            l2_reg = torch.sum(flatt[:-1]*flatt[:-1])

            valloss = 0.
            for batch_idx in list(loader_val.batch_sampler):
            
                inputs, targets = loader_val.dataset[batch_idx]
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                scores = self.model(inputs)
                valloss += self.criterion(scores, targets).item() 
            
            constraint = valloss/len(loader_val.batch_sampler) - self.delta
            multiplier = alphas*constraint #torch.dot(alphas,constraint)

            F_curr += (self.lam*l2_reg*len(curr_subset) + multiplier).item()

        val_mul = multiplier.item()
        
        #print(self.lam*l2_reg*len(curr_subset), multiplier)
        #print(F_curr)

        main_optimizer = torch.optim.Adam([{'params': self.model.parameters()}], lr=self.lr)
        #dual_optimizer = torch.optim.Adam([{'params': alphas}], lr=self.lr)

        l = [torch.flatten(p) for p in self.model.state_dict().values()]
        flat = torch.cat(l).detach()

        loader_tr = DataLoader(CustomDataset_WithId(self.x_trn[curr_subset], self.y_trn[curr_subset],\
            transform=None),shuffle=False,batch_size=self.batch_size)

        #ele_delta = self.delta.repeat(min(self.batch_size,self.y_trn[curr_subset].shape[0])).to(self.device)

        beta1,beta2 = main_optimizer.param_groups[0]['betas']
        #main_optimizer.param_groups[0]['eps']

        rem_len = (len(curr_subset)-1)

        b_idxs = 0

        device_new = self.device #"cuda:2" #self.device #

        for batch_idx in list(loader_tr.batch_sampler):

            inputs, targets, _ = loader_tr.dataset[batch_idx]
            inputs, targets = inputs.to(self.device), targets.to(self.device)

            ele_delta = self.delta.repeat(targets.shape[0]).to(self.device)
        
            weights = flat.repeat(targets.shape[0], 1)
            ele_alphas = alphas.detach().repeat(targets.shape[0]).to(self.device)

            exp_avg_w = w_exp_avg.repeat(targets.shape[0], 1)#torch.zeros_like(weights)
            exp_avg_sq_w = w_exp_avg_sq.repeat(targets.shape[0], 1) #torch.zeros_like(weights)

            exp_avg_a = a_exp_avg.repeat(targets.shape[0])#torch.zeros_like(ele_alphas)
            exp_avg_sq_a = a_exp_avg_sq.repeat(targets.shape[0]) #torch.zeros_like(ele_alphas)

            exten_inp = torch.cat((inputs,torch.ones(inputs.shape[0],device=self.device).view(-1,1)),dim=1)

            bias_correction1 = beta1**step#1.0 
            bias_correction2 = beta2**step#1.0 

            for i in range(p_epoch):

                fin_val_loss_g = torch.zeros_like(weights).to(device_new)
                #val_losses = torch.zeros_like(ele_delta).to(device_new)
                for batch_idx_val in list(loader_val.batch_sampler):
                    
                    inputs_val, targets_val = loader_val.dataset[batch_idx_val]
                    inputs_val, targets_val = inputs_val.to(self.device), targets_val.to(self.device)

                    exten_val = torch.cat((inputs_val,torch.ones(inputs_val.shape[0],device=self.device)\
                        .view(-1,1)),dim=1).to(device_new)
                    #print(exten_val.shape)

                    exten_val_y = targets_val.view(-1,1).repeat(1,min(self.batch_size,\
                        targets.shape[0])).to(device_new)
                    #print(exten_val_y[0])
                
                    val_loss_p = 2*(torch.matmul(exten_val,torch.transpose(weights, 0, 1).to(device_new))\
                         - exten_val_y) 
                    #val_losses += torch.mean(val_loss_p*val_loss_p,dim=0)
                    #val_loss_g = torch.unsqueeze(val_loss_p, dim=2).repeat(1,1,flat.shape[0])
                    #print(val_loss_g[0][0])

                    #mod_val = torch.unsqueeze(exten_val, dim=1).repeat(1,targets.shape[0],1)
                    #print(mod_val[0])
                    fin_val_loss_g += torch.mean(val_loss_p[:,:,None]*exten_val[:,None,:],dim=0)

                    del exten_val,exten_val_y,val_loss_p,inputs_val, targets_val #mod_val,val_loss_g,
                    torch.cuda.empty_cache()

                fin_val_loss_g /= len(loader_val.batch_sampler)
                fin_val_loss_g = fin_val_loss_g.to(self.device)

                sum_fin_trn_loss_g = torch.zeros_like(weights).to(device_new)
                for batch_idx_trn in list(loader_tr.batch_sampler):
                    
                    inputs_trn, targets_trn,_ = loader_tr.dataset[batch_idx_trn]
                    inputs_trn, targets_trn = inputs_trn.to(self.device), targets_trn.to(self.device)

                    exten_trn = torch.cat((inputs_trn,torch.ones(inputs_trn.shape[0]\
                        ,device=self.device).view(-1,1)),dim=1).to(device_new)
                    exten_trn_y = targets_trn.view(-1,1).repeat(1,min(self.batch_size,\
                        targets.shape[0])).to(device_new)
                    #print(exten_val_y[0])
                
                    sum_trn_loss_p = 2*(torch.matmul(exten_trn,torch.transpose(weights, 0, 1)\
                        .to(device_new)) - exten_trn_y)
                    #sum_trn_loss_g = torch.unsqueeze(trn_loss_p, dim=2).repeat(1,1,flat.shape[0])

                    #mod_trn = torch.unsqueeze(exten_trn, dim=1).repeat(1,targets.shape[0],1)
                    sum_fin_trn_loss_g += torch.sum(sum_trn_loss_p[:,:,None]*exten_trn[:,None,:],dim=0)

                    #print(sum_fin_trn_loss_g.shape)

                    del exten_trn,exten_trn_y,sum_trn_loss_p,inputs_trn, targets_trn #mod_trn,sum_trn_loss_g,
                    torch.cuda.empty_cache()

                #fin_trn_loss_g /= len(loader_tr.batch_sampler)
                sum_fin_trn_loss_g = sum_fin_trn_loss_g.to(self.device)

                trn_loss_g = torch.sum(exten_inp*weights,dim=1) - targets
                fin_trn_loss_g = exten_inp*2*trn_loss_g[:,None]

                fin_trn_loss_g = (sum_fin_trn_loss_g - fin_trn_loss_g)/rem_len

                weight_grad = fin_trn_loss_g+ 2*rem_len*\
                    torch.cat((weights[:,:-1], torch.zeros((weights.shape[0],1),device=self.device)),dim=1)+\
                        fin_val_loss_g*ele_alphas[:,None]

                #print(weight_grad[0])

                exp_avg_w.mul_(beta1).add_(1.0 - beta1, weight_grad)
                exp_avg_sq_w.mul_(beta2).addcmul_(1.0 - beta2, weight_grad, weight_grad)
                denom = exp_avg_sq_w.sqrt().add_(main_optimizer.param_groups[0]['eps'])

                bias_correction1 *= beta1
                bias_correction2 *= beta2
                step_size = (self.lr)* math.sqrt(1.0-bias_correction2) / (1.0-bias_correction1)
                weights.addcdiv_(-step_size, exp_avg_w, denom)
                
                #weights = weights - self.lr*(weight_grad)

                '''print(self.lr)
                print((fin_trn_loss_g+ 2*self.lam*weights +fin_val_loss_g*ele_alphas[:,None])[0])'''

                #print(weights[0])

                val_losses = torch.zeros_like(ele_delta).to(device_new)
                for batch_idx_val in list(loader_val.batch_sampler):
                    
                    inputs_val, targets_val = loader_val.dataset[batch_idx_val]
                    inputs_val, targets_val = inputs_val.to(self.device), targets_val.to(self.device)

                    exten_val = torch.cat((inputs_val,torch.ones(inputs_val.shape[0],device=self.device)\
                        .view(-1,1)),dim=1).to(device_new)
                    #print(exten_val.shape)

                    exten_val_y = targets_val.view(-1,1).repeat(1,min(self.batch_size,\
                        targets.shape[0])).to(device_new)
                    #print(exten_val_y[0])
                
                    val_loss_p = torch.matmul(exten_val,torch.transpose(weights, 0, 1).to(device_new))\
                         - exten_val_y #
                    val_losses += torch.mean(val_loss_p*val_loss_p,dim=0)

                    del exten_val,exten_val_y,val_loss_p,inputs_val, targets_val
                    torch.cuda.empty_cache()

                val_losses = val_losses.to(self.device)

                alpha_grad = val_losses/len(loader_val.batch_sampler)-ele_delta

                exp_avg_a.mul_(beta1).add_(1.0 - beta1, alpha_grad)
                exp_avg_sq_a.mul_(beta2).addcmul_(1.0 - beta2, alpha_grad, alpha_grad)
                denom = exp_avg_sq_a.sqrt().add_(main_optimizer.param_groups[0]['eps'])
                ele_alphas.addcdiv_(step_size, exp_avg_a, denom)
                ele_alphas[ele_alphas < 0] = 0
                #print(ele_alphas[0])

                #ele_alphas = ele_alphas + self.lr*(torch.mean(val_loss_p*val_loss_p,dim=0)-ele_delta)

            val_losses = 0.
            for batch_idx_val in list(loader_val.batch_sampler):
                    
                inputs_val, targets_val = loader_val.dataset[batch_idx_val]
                inputs_val, targets_val = inputs_val.to(self.device), targets_val.to(self.device)

                exten_val = torch.cat((inputs_val,torch.ones(inputs_val.shape[0],device=self.device).view(-1,1)),dim=1)
                exten_val_y = targets_val.view(-1,1).repeat(1,targets.shape[0])
                
                val_loss = torch.matmul(exten_val,torch.transpose(weights, 0, 1)) - exten_val_y

                val_losses+= torch.mean(val_loss*val_loss,dim=0)
            
            reg = torch.sum(weights[:,:-1]*weights[:,:-1],dim=1)

            trn_losses = 0.
            for batch_idx_trn in list(loader_tr.batch_sampler):
                    
                inputs_trn, targets_trn,_ = loader_tr.dataset[batch_idx_trn]
                inputs_trn, targets_trn = inputs_trn.to(self.device), targets_trn.to(self.device)

                exten_trn = torch.cat((inputs_trn,torch.ones(inputs_trn.shape[0],device=self.device).view(-1,1)),dim=1)
                exten_trn_y = targets_trn.view(-1,1).repeat(1,min(self.batch_size,targets.shape[0]))
                #print(exten_val_y[0])
            
                trn_loss = torch.matmul(exten_trn,torch.transpose(weights, 0, 1)) - exten_trn_y
                
                trn_losses+= torch.sum(trn_loss*trn_loss,dim=0)

            trn_loss_ind = torch.sum(exten_inp*weights,dim=1) - targets

            trn_losses -= trn_loss_ind*trn_loss_ind

            abs_value = F_curr - (trn_losses + self.lam*reg*rem_len \
                + (val_losses/len(loader_val.batch_sampler)-ele_delta)*ele_alphas) 

            neg_ind = ((abs_value ) < 0).nonzero().view(-1)

            abs_value [neg_ind] = torch.max(self.F_values)

            m_values[torch.tensor(curr_subset, dtype = torch.long)[b_idxs*self.batch_size:(b_idxs+1)*self.batch_size]]\
                 = abs_value

            b_idxs +=1

        values,indices =m_values.topk(budget,largest=False)

        return list(indices.cpu().numpy())

class FindSubset_Vect_TrnLoss(object):
    def __init__(self, x_trn, y_trn, x_val, y_val,model,loss,device,delta,lr,lam,batch):
        
        self.x_trn = x_trn
        self.y_trn = y_trn
        #self.trn_batch = trn_batch

        self.x_val = x_val
        self.y_val = y_val

        self.model = model
        self.criterion = loss 
        self.device = device

        self.delta = delta
        self.lr = lr
        self.lam = lam
        #self.optimizer = optimizer
        self.batch_size = batch

        torch.manual_seed(42)
        torch.cuda.manual_seed(42)
        np.random.seed(42)


    def precompute(self,f_pi_epoch,p_epoch,alphas):

        main_optimizer = torch.optim.Adam([
                {'params': self.model.parameters()}], lr=self.lr)
                
        dual_optimizer = torch.optim.Adam([{'params': alphas}], lr=self.lr)

        print("starting Pre compute")
        #alphas = torch.rand_like(self.delta,requires_grad=True) 

        #print(alphas)

        #scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=[10,20,40,100],\
        #    gamma=0.5) #[e*2 for e in change]

        #alphas.requires_grad = False
        loader_val = DataLoader(CustomDataset(self.x_val, self.y_val,transform=None),\
            shuffle=False,batch_size=self.batch_size)
            

        #Compute F_phi
        #for i in range(f_pi_epoch):

        prev_loss = 1000
        stop_count = 0
        i=0
        
        while(True):
            
            main_optimizer.zero_grad()
            
            '''l2_reg = 0
            for param in self.model.parameters():
                l2_reg += torch.norm(param)'''

            #l = [torch.flatten(p) for p in main_model.parameters()]
            #flat = torch.cat(l)
            #l2_reg = torch.sum(flat*flat)
            
            constraint = 0. 
            for batch_idx in list(loader_val.batch_sampler):
                    
                inputs, targets = loader_val.dataset[batch_idx]
                inputs, targets = inputs.to(self.device), targets.to(self.device)
            
                val_out = self.model(inputs)
                constraint += self.criterion(val_out, targets)
                
            constraint /= len(loader_val.batch_sampler)
            constraint = constraint - self.delta
            multiplier = alphas*constraint #torch.dot(alphas,constraint)

            loss = multiplier
            self.F_phi = loss.item()
            loss.backward()
            
            main_optimizer.step()
            #scheduler.step()
            
            '''for param in self.model.parameters():
                param.requires_grad = False
            alphas.requires_grad = True'''

            dual_optimizer.zero_grad()

            constraint = 0.
            for batch_idx in list(loader_val.batch_sampler):
                    
                inputs, targets = loader_val.dataset[batch_idx]
                inputs, targets = inputs.to(self.device), targets.to(self.device)
            
                val_out = self.model(inputs)
                constraint += self.criterion(val_out, targets)
            
            constraint /= len(loader_val.batch_sampler)
            constraint = constraint - self.delta
            multiplier = -1.0*alphas*constraint #torch.dot(-1.0*alphas,constraint)
            
            multiplier.backward()
            
            dual_optimizer.step()

            alphas.requires_grad = False
            alphas.clamp_(min=0.0)
            alphas.requires_grad = True
            #print(alphas)

            '''for param in self.model.parameters():
                param.requires_grad = True'''

            if loss.item() <= 0.:
                break

            #if i>= f_pi_epoch:
            #    break

            if abs(prev_loss - loss.item()) <= 1e-3 and stop_count >= 5:
                break 
            elif abs(prev_loss - loss.item()) <= 1e-3:
                stop_count += 1
            else:
                stop_count = 0

            prev_loss = loss.item()
            i+=1

            #if i % 50 == 0:
            #    print(loss.item(),alphas,constraint)

        print("Finishing F phi")

        if loss.item() <= 0.:
            alphas = torch.zeros_like(alphas)

        #print(loss.item())

        l = [torch.flatten(p) for p in self.model.state_dict().values()]
        flat = torch.cat(l).detach().clone()
        
        self.F_values = torch.zeros(len(self.x_trn),device=self.device)

        device_new = self.device #"cuda:2"#self.device #
        beta1,beta2 = main_optimizer.param_groups[0]['betas']
        #main_optimizer.param_groups[0]['eps']

        loader_tr = DataLoader(CustomDataset_WithId(self.x_trn, self.y_trn,\
            transform=None), device = self.device, shuffle=False,batch_size=self.batch_size*20)

        loader_val = DataLoader(CustomDataset(self.x_val, self.y_val,device = self.device,transform=None),\
            shuffle=False,batch_size=self.batch_size*20)
        
        for batch_idx in list(loader_tr.batch_sampler):

            inputs, targets, idxs = loader_tr.dataset[batch_idx]
            inputs, targets = inputs.to(self.device), targets.to(self.device)

            ele_delta = self.delta.repeat(targets.shape[0]).to(self.device)
        
            weights = flat.view(1,-1).repeat(targets.shape[0], 1)
            ele_alphas = alphas.detach().repeat(targets.shape[0]).to(self.device)
            #print(weights.shape)

            exp_avg_w = torch.zeros_like(weights)
            exp_avg_sq_w = torch.zeros_like(weights)

            #exp_avg_a = torch.zeros_like(ele_alphas)
            #exp_avg_sq_a = torch.zeros_like(ele_alphas)

            exten_inp = torch.cat((inputs,torch.ones(inputs.shape[0],device=self.device).view(-1,1))\
                ,dim=1)

            bias_correction1 = 1.0 
            bias_correction2 = 1.0 

            for i in range(p_epoch):

                trn_loss_g = torch.sum(exten_inp*weights,dim=1) - targets
                fin_trn_loss_g = exten_inp*2*trn_loss_g[:,None]

                #no_bias = weights.clone()
                #no_bias[-1,:] = torch.zeros(weights.shape[0])
                
                weight_grad = fin_trn_loss_g+ 2*self.lam*\
                    torch.cat((weights[:,:-1], torch.zeros((weights.shape[0],1),device=self.device)),dim=1)
                #+fin_val_loss_g*ele_alphas[:,None]

                exp_avg_w.mul_(beta1).add_(1.0 - beta1, weight_grad)
                exp_avg_sq_w.mul_(beta2).addcmul_(1.0 - beta2, weight_grad, weight_grad)
                denom = exp_avg_sq_w.sqrt().add_(main_optimizer.param_groups[0]['eps'])

                bias_correction1 *= beta1
                bias_correction2 *= beta2
                step_size = (self.lr) * math.sqrt(1.0-bias_correction2) / (1.0-bias_correction1)
                weights.addcdiv_(-step_size, exp_avg_w, denom)
                
            val_losses = 0.
            for batch_idx_val in list(loader_val.batch_sampler):
                    
                inputs_val, targets_val = loader_val.dataset[batch_idx_val]
                inputs_val, targets_val = inputs_val.to(self.device), targets_val.to(self.device)

                exten_val = torch.cat((inputs_val,torch.ones(inputs_val.shape[0],device=self.device).view(-1,1)),dim=1)
                #exten_val_y = targets_val.view(-1,1).repeat(1,min(self.batch_size*20,targets.shape[0]))
                
                exten_val_y = torch.mean(targets_val).repeat(min(self.batch_size*20,targets.shape[0]))

                #val_loss = torch.matmul(torch.mean(exten_val,dim=0),weights.T).view(-1) - exten_val_y 
                #torch.transpose(weights, 0, 1)

                val_loss = torch.sum(weights*torch.mean(exten_val,dim=0),dim=1) - exten_val_y

                val_losses+= val_loss*val_loss #torch.mean(val_loss*val_loss,dim=0)
            
            reg = torch.sum(weights[:,:-1]*weights[:,:-1],dim=1)
            trn_loss = torch.sum(exten_inp*weights,dim=1) - targets

            #print(torch.sum(exten_inp*weights,dim=1)[0])
            #print((trn_loss*trn_loss)[0],self.lam*reg[0],\
            #    ((torch.mean(val_loss*val_loss,dim=0)-ele_delta)*ele_alphas)[0])

            self.F_values[idxs] = trn_loss*trn_loss+ self.lam*reg +torch.max(torch.zeros_like(ele_alphas),\
                (val_losses/len(loader_val.batch_sampler)-ele_delta)*ele_alphas)

        #print(self.F_values[:10])

        #self.F_values = self.F_values - max(loss.item(),0.) 

        print("Finishing Element wise F")


    def return_subset(self,theta_init,p_epoch,curr_subset,budget,batch,\
        step,w_exp_avg,w_exp_avg_sq):#,alphas,a_exp_avg,a_exp_avg_sq):

        m_values = self.F_values.detach().clone() #torch.zeros(len(self.x_trn))
        
        self.model.load_state_dict(theta_init)

        #print(theta_init)
        #print(curr_subset)

        '''main_optimizer = torch.optim.Adam([
                {'params': self.model.parameters()}], lr=self.lr)
                
        dual_optimizer = torch.optim.Adam([{'params': alphas}], lr=self.lr)'''

        loader_tr = DataLoader(CustomDataset_WithId(self.x_trn[curr_subset], self.y_trn[curr_subset],\
            transform=None),shuffle=False,batch_size=batch)

        #loader_val = DataLoader(CustomDataset(self.x_val, self.y_val,transform=None),\
        #    shuffle=False,batch_size=batch)  

        sum_error = torch.nn.MSELoss(reduction='sum')       

        with torch.no_grad():

            F_curr = 0.

            for batch_idx in list(loader_tr.batch_sampler):
            
                inputs, targets, _ = loader_tr.dataset[batch_idx]
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                scores = self.model(inputs)
                #print(self.criterion(scores, targets).item())

                F_curr += sum_error(scores, targets).item() 

            #F_curr /= len(loader_tr.batch_sampler)
            #print(F_curr,end=",")

            l = [torch.flatten(p) for p in self.model.parameters()]
            flatt = torch.cat(l)
            l2_reg = torch.sum(flatt[:-1]*flatt[:-1])

            F_curr += (self.lam*l2_reg*len(curr_subset)).item() #+ multiplier).item()

        #val_mul = multiplier.item()
        
        #print(self.lam*l2_reg*len(curr_subset))#, multiplier)
        #print(F_curr)

        main_optimizer = torch.optim.Adam([{'params': self.model.parameters()}], lr=self.lr)
        #dual_optimizer = torch.optim.Adam([{'params': alphas}], lr=self.lr)

        l = [torch.flatten(p) for p in self.model.state_dict().values()]
        flat = torch.cat(l).detach()

        loader_tr = DataLoader(CustomDataset_WithId(self.x_trn[curr_subset], self.y_trn[curr_subset],\
            transform=None),shuffle=False,batch_size=self.batch_size)

        #ele_delta = self.delta.repeat(min(self.batch_size,self.y_trn[curr_subset].shape[0])).to(self.device)

        beta1,beta2 = main_optimizer.param_groups[0]['betas']
        #main_optimizer.param_groups[0]['eps']

        rem_len = (len(curr_subset)-1)

        b_idxs = 0

        device_new = self.device #"cuda:2" #self.device #

        for batch_idx in list(loader_tr.batch_sampler):

            inputs, targets, _ = loader_tr.dataset[batch_idx]
            inputs, targets = inputs.to(self.device), targets.to(self.device)

            #ele_delta = self.delta.repeat(targets.shape[0]).to(self.device)
        
            weights = flat.repeat(targets.shape[0], 1)
            #ele_alphas = alphas.detach().repeat(targets.shape[0]).to(self.device)

            exp_avg_w = w_exp_avg.repeat(targets.shape[0], 1)#torch.zeros_like(weights)
            exp_avg_sq_w = w_exp_avg_sq.repeat(targets.shape[0], 1) #torch.zeros_like(weights)

            #exp_avg_a = a_exp_avg.repeat(targets.shape[0])#torch.zeros_like(ele_alphas)
            #exp_avg_sq_a = a_exp_avg_sq.repeat(targets.shape[0]) #torch.zeros_like(ele_alphas)

            exten_inp = torch.cat((inputs,torch.ones(inputs.shape[0],device=self.device).view(-1,1)),dim=1)

            bias_correction1 = beta1**step#1.0 
            bias_correction2 = beta2**step#1.0 

            for i in range(p_epoch):

                sum_fin_trn_loss_g = torch.zeros_like(weights).to(device_new)
                for batch_idx_trn in list(loader_tr.batch_sampler):
                    
                    inputs_trn, targets_trn,_ = loader_tr.dataset[batch_idx_trn]
                    inputs_trn, targets_trn = inputs_trn.to(self.device), targets_trn.to(self.device)

                    exten_trn = torch.cat((inputs_trn,torch.ones(inputs_trn.shape[0]\
                        ,device=self.device).view(-1,1)),dim=1).to(device_new)
                    exten_trn_y = targets_trn.view(-1,1).repeat(1,min(self.batch_size,\
                        targets.shape[0])).to(device_new)
                    #print(exten_val_y[0])
                
                    sum_trn_loss_p = 2*(torch.matmul(exten_trn,torch.transpose(weights, 0, 1)\
                        .to(device_new)) - exten_trn_y)
                    #sum_trn_loss_g = torch.unsqueeze(trn_loss_p, dim=2).repeat(1,1,flat.shape[0])

                    #mod_trn = torch.unsqueeze(exten_trn, dim=1).repeat(1,targets.shape[0],1)
                    sum_fin_trn_loss_g += torch.sum(sum_trn_loss_p[:,:,None]*exten_trn[:,None,:],dim=0)

                    #print(sum_fin_trn_loss_g.shape)

                    del exten_trn,exten_trn_y,sum_trn_loss_p,inputs_trn, targets_trn #mod_trn,sum_trn_loss_g,
                    torch.cuda.empty_cache()

                #fin_trn_loss_g /= len(loader_tr.batch_sampler)
                sum_fin_trn_loss_g = sum_fin_trn_loss_g.to(self.device)

                trn_loss_g = torch.sum(exten_inp*weights,dim=1) - targets
                fin_trn_loss_g = exten_inp*2*trn_loss_g[:,None]

                fin_trn_loss_g = (sum_fin_trn_loss_g - fin_trn_loss_g)/rem_len

                weight_grad = fin_trn_loss_g+ 2*rem_len*\
                    torch.cat((weights[:,:-1], torch.zeros((weights.shape[0],1),device=self.device)),dim=1)#+\
                #fin_val_loss_g*ele_alphas[:,None]

                #print(weight_grad[0])

                exp_avg_w.mul_(beta1).add_(1.0 - beta1, weight_grad)
                exp_avg_sq_w.mul_(beta2).addcmul_(1.0 - beta2, weight_grad, weight_grad)
                denom = exp_avg_sq_w.sqrt().add_(main_optimizer.param_groups[0]['eps'])

                bias_correction1 *= beta1
                bias_correction2 *= beta2
                step_size = (self.lr)* math.sqrt(1.0-bias_correction2) / (1.0-bias_correction1)
                weights.addcdiv_(-step_size, exp_avg_w, denom)
            
            reg = torch.sum(weights[:,:-1]*weights[:,:-1],dim=1)

            trn_losses = 0.
            for batch_idx_trn in list(loader_tr.batch_sampler):
                    
                inputs_trn, targets_trn,_ = loader_tr.dataset[batch_idx_trn]
                inputs_trn, targets_trn = inputs_trn.to(self.device), targets_trn.to(self.device)

                exten_trn = torch.cat((inputs_trn,torch.ones(inputs_trn.shape[0],device=self.device).view(-1,1)),dim=1)
                exten_trn_y = targets_trn.view(-1,1).repeat(1,min(self.batch_size,targets.shape[0]))
                #print(exten_val_y[0])
            
                trn_loss = torch.matmul(exten_trn,torch.transpose(weights, 0, 1)) - exten_trn_y
                
                trn_losses+= torch.sum(trn_loss*trn_loss,dim=0)

            trn_loss_ind = torch.sum(exten_inp*weights,dim=1) - targets

            trn_losses -= trn_loss_ind*trn_loss_ind
            
            abs_value = F_curr - (trn_losses + self.lam*reg*rem_len) #\
            
            neg_ind = ((abs_value ) < 0).nonzero().view(-1)

            abs_value [neg_ind] = torch.max(self.F_values)

            m_values[torch.tensor(curr_subset)[b_idxs*self.batch_size:(b_idxs+1)*self.batch_size]]\
                 = abs_value

            b_idxs +=1

        values,indices =m_values.topk(budget,largest=False)

        return list(indices.cpu().numpy())


class SetFunctionFacLoc(object):

    def __init__(self, device, train_full_loader):#, valid_loader):
        
        self.train_loader = train_full_loader      
        self.device = device #torch.device("cuda" if torch.cuda.is_available() else "cpu")
    

    def distance(self,x, y, exp = 2):

      n = x.size(0)
      m = y.size(0)
      d = x.size(1)

      x = x.unsqueeze(1).expand(n, m, d)
      y = y.unsqueeze(0).expand(n, m, d)

      dist = torch.pow(x - y, exp).sum(2) 
      return dist 

    def compute_score(self):
      self.N = 0
      g_is = []#self.train_loader

      with torch.no_grad():
        for i, data_i in  enumerate(self.train_loader, 0):
          inputs_i, target_i = data_i
          inputs_i = inputs_i.to(self.device) #, target_i.to(self.device)
          self.N += inputs_i.size()[0]
          g_is.append(inputs_i)
        
        self.sim_mat = torch.zeros([self.N, self.N],dtype=torch.float32)

        first_i = True

        for i, g_i in enumerate(g_is, 0):

          if first_i:
            size_b = g_i.size(0)
            first_i = False

          for j, g_j in enumerate(g_is, 0):
            self.sim_mat[i*size_b: i*size_b + g_i.size(0), j*size_b: j*size_b + g_j.size(0)] = self.distance(g_i, g_j)
      self.const = torch.max(self.sim_mat).item()
      self.sim_mat = self.const - self.sim_mat
      #self.sim_mat = self.sim_mat.to(self.device)
      dist = self.sim_mat.sum(1)
      bestId = torch.argmax(dist).item()
      self.max_sim = self.sim_mat[bestId].to(self.device)
      return bestId


    def lazy_greedy_max(self, budget,logfile):
      
      #starting = time.process_time() 
      id_first = self.compute_score()
      self.gains = PriorityQueue()
      for i in range(self.N):
        if i == id_first :
          continue
        curr_gain = (torch.max(self.max_sim ,self.sim_mat[i].to(self.device)) - self.max_sim).sum()
        self.gains.put((-curr_gain.item(),i))

      numSelected = 2
      second = self.gains.get()
      greedyList = [id_first, second[1]]
      self.max_sim = torch.max(self.max_sim,self.sim_mat[second[1]].to(self.device))

      #ending = time.process_time()
      #print("Kernel computation time ",ending-starting, file=logfile)

      starting = time.process_time()
      while(numSelected < budget):

          if self.gains.empty():
            break

          elif self.gains.qsize() == 1:
            bestId = self.gains.get()[1]

          else:
   
            bestGain = -np.inf
            bestId = None
            
            while True:

              first =  self.gains.get()

              if bestId == first[1]: 
                break

              curr_gain = (torch.max(self.max_sim, self.sim_mat[first[1]].to(self.device)) - self.max_sim).sum()
              self.gains.put((-curr_gain.item(), first[1]))


              if curr_gain.item() >= bestGain:
                  
                bestGain = curr_gain.item()
                bestId = first[1]

          greedyList.append(bestId)
          numSelected += 1

          self.max_sim = torch.max(self.max_sim,self.sim_mat[bestId].to(self.device))

      #print()
      #gamma = self.compute_gamma(greedyList)

      #ending = time.process_time()
      #print("Selectio time ",ending-starting, file=logfile)

      return greedyList



def process_time_series(datadir, past_length,col_name,name,save_data=True):

    x_trn = []
    y_trn = []

    path = os.path.join(datadir, 'prices-split-adjusted.csv')

    prices_split_adjusted = pd.read_csv(path,index_col='date', parse_dates=['date'])

    #Removing data with less than 1500 entries
    symbol = pd.DataFrame(prices_split_adjusted['symbol'].value_counts())
    omit = symbol[symbol['symbol'] < 1500].index
    prices_split_adjusted = prices_split_adjusted[~prices_split_adjusted['symbol'].isin(omit)]

    symbol = pd.DataFrame(prices_split_adjusted['symbol'].value_counts())

    for sym in symbol.index:

        stock = prices_split_adjusted[prices_split_adjusted['symbol'] == sym]

        data_len = len(stock.index)
        
        x_trn_slice = np.zeros((data_len-past_length,past_length))
        y_trn_slice = np.zeros(data_len-past_length)
        
        for ind in range(data_len-past_length):
            x_trn_slice[ind] = stock[col_name][stock.index[ind:ind+past_length]]
            y_trn_slice[ind] = stock[col_name][stock.index[ind+past_length]]

        x_trn.append(x_trn_slice)
        y_trn.append(y_trn_slice)

    finaL_x_trn = np.concatenate(x_trn, axis=0)
    finaL_y_trn = np.concatenate(y_trn, axis=0)

    
    if save_data:
        # Save the numpy files to the folder where they come from
        data_np_path = os.path.join(datadir, name+'_'+str(past_length)+'.data.npy') 
        target_np_path = os.path.join(datadir, name+'_'+str(past_length)+'.label.npy') 
        np.save(data_np_path, finaL_x_trn)
        np.save(target_np_path, finaL_y_trn)

    return finaL_x_trn, finaL_y_trn



def load_time_series_data (datadir, dset_name,past_length,clean=True):

    if os.path.isfile(os.path.join(datadir, dset_name+'_'+str(past_length)+'.data.npy')) and \
            os.path.isfile(os.path.join(datadir, dset_name+'_'+str(past_length)+'.label.npy')):
        
        x_trn = np.load(os.path.join(datadir, dset_name+'_'+str(past_length)+'.data.npy'))
        y_trn  = np.load(os.path.join(datadir, dset_name+'_'+str(past_length)+'.label.npy'))

    else:
    
        if dset_name == "NY_Stock_exchange_close":
            x_trn, y_trn  = process_time_series(datadir,past_length,'close',dset_name)

        elif dset_name == "NY_Stock_exchange_open":
            x_trn, y_trn  = process_time_series(datadir,past_length,'open',dset_name)

        elif dset_name == "NY_Stock_exchange_high":
            x_trn, y_trn  = process_time_series(datadir,past_length,'high',dset_name)

        elif dset_name == "NY_Stock_exchange_low":
            x_trn, y_trn  = process_time_series(datadir,past_length,'low',dset_name)

    y_trn = y_trn #+ 1600
    
    x_trn, x_tst, y_trn, y_tst = train_test_split(x_trn, y_trn, test_size=0.1, random_state=42)
    x_trn, x_val, y_trn, y_val = train_test_split(x_trn, y_trn, test_size=0.005, random_state=42)
    
    if not clean:

        noise_size = int(len(y_trn) * 0.5)
        noise_indices = np.random.choice(np.arange(len(y_trn)), size=noise_size, replace=False)
        
        sigma = 40
        y_trn[noise_indices] = y_trn[noise_indices] + np.random.normal(0, sigma, noise_size)
    
    sc = MinMaxScaler() #StandardScaler()
    x_trn = sc.fit_transform(x_trn)
    x_val = sc.transform(x_val)
    x_tst = sc.transform(x_tst)

    '''sc_l = MinMaxScaler() #StandardScaler()
    y_trn = np.reshape(sc_l.fit_transform(np.reshape(y_trn,(-1,1))),(-1))'''
    #y_val = np.reshape(sc_l.fit_transform(np.reshape(y_val,(-1,1))),(-1))
    #y_tst = np.reshape(sc_l.fit_transform(np.reshape(y_tst,(-1,1))),(-1))

    fullset = (x_trn, y_trn)
    valset = (x_val, y_val)
    testset = (x_tst, y_tst)

    return fullset, valset, testset #, sc_l




def run_stochastic_Facloc( data, targets, sub_budget, budget,logfile,device='cpu'):
    

    num_iterations = int(math.ceil(len(data)/sub_budget))
    trn_indices = list(np.arange(len(data)))
    facloc_indices = []

    per_iter_bud = int(budget/num_iterations)

    trn_batch = 1200
 
    for i in range(num_iterations):
        
        rem_indices = list(set(trn_indices).difference(set(facloc_indices)))

        state = np.random.get_state()
        np.random.seed(i*i)
        sub_indices = np.random.choice(rem_indices, size=sub_budget, replace=False)
        np.random.set_state(state)

        data_subset = data[sub_indices].cpu()
        targets_subset = targets[sub_indices].cpu()
        train_loader_greedy = []
        for item in range(math.ceil(sub_budget /trn_batch)):
          inputs = data_subset[item*trn_batch:(item+1)*trn_batch]
          target  = targets_subset[item*trn_batch:(item+1)*trn_batch]
          train_loader_greedy.append((inputs,target))
        
        #train_loader_greedy.append((data_subset, targets_subset))
        setf_model = SetFunctionFacLoc(device, train_loader_greedy)
        idxs = setf_model.lazy_greedy_max(min(per_iter_bud,budget-len(facloc_indices)),logfile)#, model)
        facloc_indices.extend([sub_indices[idx] for idx in idxs])
    return facloc_indices


class RegressionNet(nn.Module):
    def __init__(self, input_dim):
        super(RegressionNet, self).__init__()
        self.linear = nn.Linear(input_dim, 1)#,bias=False)
        self.feature_dim = input_dim
    
    def forward(self, x, last=False):
        scores = self.linear(x)
        #scores = torch.sigmoid(self.linear(x))
        #return scores.view(-1)
        if last:
            return scores.view(-1), x
        else:
            return scores.view(-1)

    def get_embedding_dim(self):
        return self.feature_dim

class LogisticNet(nn.Module):
    def __init__(self, input_dim):#,num_cls):
        super(LogisticNet, self).__init__()
        self.linear = nn.Linear(input_dim,1)
        #self.feature_dim = input_dim
    
    def forward(self, x, last=False):
        scores = self.linear(x)
        scores = torch.sigmoid(self.linear(x).view(-1))
        return scores



def get_slices(data_name, data,labels,device,buckets=None,clean=True):

    #data_slices = []
    #abel_slices =[]
    
    val_data_slices = []
    val_label_slices =[]

    tst_data_slices = []
    tst_label_slices =[]

    '''sc = StandardScaler()
    x_trn = sc.fit_transform(x_trn)
    x_val = sc.transform(x_val)
    x_tst = sc.transform(x_tst)'''

    if data_name == 'Community_Crime_old':
        protect_feature =[2,3,4,5]

        data_class =[]

        N = int(0.1*len(data)/(buckets*len(protect_feature)))

        total_set = set(list(np.arange(len(data))))
        
        for i in protect_feature:            

            digit = np.ones(data.shape[0],dtype=np.int8)*(i-1)
            low = np.min(data[:,i])
            high = np.max(data[:,i])
            bins = np.linspace(low, high, buckets)
            digitized = np.digitize(data[:,i], bins)
            digit =  digit*10 + digitized

            classes,times = np.unique(digit,return_counts=True) 
            times, classes = zip(*sorted(zip(times, classes)))
            data_class.append(classes)
            
            count = 0
            for cl in classes[:-1]:

                indices=[]
                indices_tst=[]
                
                idx = (digit == cl).nonzero()[0].flatten()
                idx.tolist()
                idxs = set(idx)
                idxs.intersection_update(total_set)
                idx = list(idxs)
                #print(cl,len(idx))

                curr_N = int(len(idx)/3)

                #print(curr_N,N)
                 
                indices.extend(list(np.random.choice(idx, size=min(N,curr_N), replace=False)))
                total_set.difference(indices)
                idxs.difference(indices)
                idx = list(idxs)
                indices_tst.extend(list(np.random.choice(idx, size=min(N,curr_N), replace=False)))
                total_set.difference(indices_tst)
                
                if curr_N < N:
                    count += (N - curr_N)

                val_data_slices.append(torch.from_numpy(data[indices]).float().to(device))
                val_label_slices.append(torch.from_numpy(labels[indices]).float().to(device))

                tst_data_slices.append(torch.from_numpy(data[indices_tst]).float().to(device))
                tst_label_slices.append(torch.from_numpy(labels[indices_tst]).float().to(device))

            indices=[]
            indices_tst=[]
            
            idx = (digit == classes[-1]).nonzero()[0].flatten()
            idx.tolist()
            idxs = set(idx)
            idxs.intersection_update(total_set)
            idx = list(idxs)

            indices.extend(list(np.random.choice(idx, size=N+count, replace=False)))
            total_set.difference(indices)
            idxs.difference(indices)
            idx = list(idxs)
            indices_tst.extend(list(np.random.choice(idx, size=N+count, replace=False)))
            total_set.difference(indices_tst)

            val_data_slices.append(torch.from_numpy(data[indices]).float().to(device))
            val_label_slices.append(torch.from_numpy(labels[indices]).float().to(device))

            tst_data_slices.append(torch.from_numpy(data[indices_tst]).float().to(device))
            tst_label_slices.append(torch.from_numpy(labels[indices_tst]).float().to(device))

        final_lables = [j for sub in data_class for j in sub]
        left = list(total_set)    
        data_left = data[left]
        label_left = labels[left]

    elif data_name == 'OnlineNewsPopularity':

        protect_feature = [11,12,13,14,15,16]

        final_lables = [ 'Lifestyle','Entertainment','Business','Social Media','Tech','World']

        total_set = set(list(np.arange(len(data))))

        N = int(0.1*len(data)/len(protect_feature))

        max_times = 0
        
        for pf in protect_feature:            
        
            classes,times = np.unique(data[:,pf],return_counts=True) 

            one_id = (classes == 1.0).nonzero()[0].flatten()[0]

            if max_times < times[one_id]:
                max_times = times[one_id]
                max_id = pf

        most = final_lables[max_id-protect_feature[0]]
        final_lables.remove(most)
        final_lables.append(most)

        count = 0
        
        for pf in protect_feature:

            if pf == max_id:
                continue
            
            idx = (data[:,pf] == 1.0).nonzero()[0].flatten()
            idx.tolist()
            idxs = set(idx)
            idxs.intersection_update(total_set)
            idx = list(idxs)
            #print(cl,len(idx))

            curr_N = int(len(idx)/3)

            #print(curr_N,N)
            
            indices = list(np.random.choice(idx, size=min(N,curr_N), replace=False))
            total_set.difference(indices)
            idxs.difference(indices)
            idx = list(idxs)
            indices_tst = list(np.random.choice(idx, size=min(N,curr_N), replace=False))
            total_set.difference(indices_tst)
            
            if curr_N < N:
                count += (N - curr_N)

            '''val_data_slices.append(torch.from_numpy(data[indices]).float().to(device))
            val_label_slices.append(torch.from_numpy(labels[indices]).float().to(device))

            tst_data_slices.append(torch.from_numpy(data[indices_tst]).float().to(device))
            tst_label_slices.append(torch.from_numpy(labels[indices_tst]).float().to(device))'''

            val_data_slices.append(data[indices])
            val_label_slices.append(labels[indices])

            tst_data_slices.append(data[indices_tst])
            tst_label_slices.append(labels[indices_tst])

            
        idx = (data[:,max_id] == 1.0).nonzero()[0].flatten()
        idx.tolist()
        idxs = set(idx)
        idxs.intersection_update(total_set)
        idx = list(idxs)

        indices = list(np.random.choice(idx, size=N+count, replace=False))
        total_set.difference(indices)
        idxs.difference(indices)
        idx = list(idxs)
        indices_tst = list(np.random.choice(idx, size=N+count, replace=False)) 
        total_set.difference(indices_tst)

        '''val_data_slices.append(torch.from_numpy(data[indices]).float().to(device))
        val_label_slices.append(torch.from_numpy(labels[indices]).float().to(device))

        tst_data_slices.append(torch.from_numpy(data[indices_tst]).float().to(device))
        tst_label_slices.append(torch.from_numpy(labels[indices_tst]).float().to(device))'''

        val_data_slices.append(data[indices])
        val_label_slices.append(labels[indices])

        tst_data_slices.append(data[indices_tst])
        tst_label_slices.append(labels[indices_tst])

        left = list(total_set)
        sc = MinMaxScaler() #StandardScaler()
        sc_l = MinMaxScaler()

        #print(data[left][0])
        data_left = sc.fit_transform(data[left])
        label_left = np.reshape(sc_l.fit_transform(np.reshape(labels[left],(-1,1))),(-1))
        #print(data_left[0])
        #preprocessing.normalize(data[left])

        for j in range(len(val_data_slices)):
            
            val_data_slices[j] = torch.from_numpy(sc.transform(val_data_slices[j])).float().to(device)
            tst_data_slices[j] = torch.from_numpy(sc.transform(tst_data_slices[j])).float().to(device)

            val_label_slices[j] = torch.from_numpy(np.reshape(\
                sc_l.transform(np.reshape(val_label_slices[j],(-1,1))),(-1))).float().to(device)
            tst_label_slices[j] = torch.from_numpy(np.reshape(\
                sc_l.transform(np.reshape(tst_label_slices[j],(-1,1))),(-1))).float().to(device)
    
    elif data_name in ['census','LawSchool','German_credit','Community_Crime']:
        
        if data_name == 'census':
            protect_feature = 8 #9
        elif data_name == 'LawSchool':
            protect_feature = 0
        elif data_name == 'German_credit':
            protect_feature = 8
        elif data_name == 'Community_Crime':
            protect_feature = -1

        total_set = set(list(np.arange(len(data))))
        
        classes,times = np.unique(data[:,protect_feature],return_counts=True) 
        times, classes = zip(*sorted(zip(times, classes)))

        #print(times)
        #print(classes)

        N = int(0.1*len(data)/len(classes))
        
        count = 0
        for cl in classes[:-1]:

            indices=[]
            indices_tst=[]
            
            idx = (data[:,protect_feature] == cl).nonzero()[0].flatten()
            idx.tolist()

            curr_N = int(len(idx)/3)

            #print(curr_N,N)
                
            indices.extend(list(np.random.choice(idx, size=min(N,curr_N), replace=False)))
            total_set.difference(indices)
            idxs = set(idx)
            idxs.difference(indices)
            idx = list(idxs)
            indices_tst.extend(list(np.random.choice(idx, size=min(N,curr_N), replace=False)))
            total_set.difference(indices_tst)
            
            if curr_N < N:
                count += (N - curr_N)

            #val_data_slices.append(torch.from_numpy(preprocessing.normalize(data[indices])).float().to(device))
            val_data_slices.append(torch.from_numpy(data[indices]).float().to(device))
            val_label_slices.append(torch.from_numpy(labels[indices]).float().to(device))

            #tst_data_slices.append(torch.from_numpy(preprocessing.normalize(data[indices_tst])).float().to(device))
            tst_data_slices.append(torch.from_numpy(data[indices_tst]).float().to(device))
            tst_label_slices.append(torch.from_numpy(labels[indices_tst]).float().to(device))

        indices=[]
        indices_tst=[]
        
        idx = (data[:,protect_feature] == classes[-1]).nonzero()[0].flatten()
        idx.tolist()

        indices.extend(list(np.random.choice(idx, size=N+count, replace=False)))
        total_set.difference(indices)
        idxs = set(idx)
        idxs.difference(indices)
        idx = list(idxs)
        indices_tst.extend(list(np.random.choice(idx, size=N+count, replace=False)))
        total_set.difference(indices_tst)

        #val_data_slices.append(torch.from_numpy(preprocessing.normalize(data[indices])).float().to(device))
        val_data_slices.append(torch.from_numpy(data[indices]).float().to(device))
        val_label_slices.append(torch.from_numpy(labels[indices]).float().to(device))

        #tst_data_slices.append(torch.from_numpy(preprocessing.normalize(data[indices_tst])).float().to(device))
        tst_data_slices.append(torch.from_numpy(data[indices_tst]).float().to(device))
        tst_label_slices.append(torch.from_numpy(labels[indices_tst]).float().to(device))

        final_lables = classes
        left = list(total_set)
        data_left = data[left] #preprocessing.normalize(data[left]) 
        label_left = labels[left]

        if not clean:

            noise_size = int(len(label_left) * 0.5)
            noise_indices = np.random.choice(np.arange(len(label_left)), size=noise_size, replace=False)
            
            sigma = 40
            label_left[noise_indices] = label_left[noise_indices] + np.random.normal(0, sigma, noise_size)
    
        
    return data_left, label_left, val_data_slices, val_label_slices, final_lables, tst_data_slices,\
        tst_label_slices,final_lables


class CustomDataset_WithId(Dataset):
    def __init__(self, data, target, transform=None):       
        self.transform = transform
        self.data = data #.astype('float32')
        self.targets = target
        self.X = self.data
        self.Y = self.targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()        
        sample_data = self.data[idx]
        label = self.targets[idx]
        if self.transform is not None:
            sample_data = self.transform(sample_data)
        return sample_data, label,idx #.astype('float32')

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                # convert float NaN --> string NaN
                output[col] = output[col].fillna('NaN')
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

## Utility function to load datasets from libsvm datasets
def csv_file_load(path,dim,skip=False,save_data=False):
    data = []
    target = []
    with open(path) as fp:
       if skip:
           line = fp.readline()
       line = fp.readline()
       while line:
        temp = [i for i in line.strip().split(",")]
        target.append(int(float(temp[-1]))) # Class Number. # Not assumed to be in (0, K-1)
        temp_data = [0]*dim
        count = 0
        for i in temp[:-1]:
            #ind, val = i.split(':')
            temp_data[count] = float(i)
            count += 1
        data.append(temp_data)
        line = fp.readline()
    X_data = np.array(data, dtype=np.float32)
    Y_label = np.array(target)
    if save_data:
        # Save the numpy files to the folder where they come from
        data_np_path = path + '.data.npy'
        target_np_path = path + '.label.npy'
        np.save(data_np_path, X_data)
        np.save(target_np_path, Y_label)
    return (X_data, Y_label)

def libsvm_file_load(path,dim, save_data=False):
    data = []
    target = []
    with open(path) as fp:
       line = fp.readline()
       while line:
        temp = [i for i in line.strip().split(" ")]
        target.append(int(float(temp[0]))) # Class Number. # Not assumed to be in (0, K-1)
        temp_data = [0]*dim
        
        for i in temp[1:]:
            if len(i) > 1: 
                ind,val = i.split(':')
                temp_data[int(ind)-1] = float(val)
        data.append(temp_data)
        line = fp.readline()
    X_data = np.array(data,dtype=np.float32)
    Y_label = np.array(target)
    if save_data:
        # Save the numpy files to the folder where they come from
        data_np_path = path + '.data.npy'
        target_np_path = path + '.label.npy'
        np.save(data_np_path, X_data)
        np.save(target_np_path, Y_label)
    return (X_data, Y_label)

def clean_lawschool_full(path):
   
    df = pd.read_csv(path)
    df = df.dropna()
    # remove y from df
    y = df['ugpa']
    y = y / 4
    df = df.drop('ugpa', 1)
    # convert gender variables to 0,1
    df['gender'] = df['gender'].map({'male': 1, 'female': 0})
    # add bar1 back to the feature set
    df_bar = df['bar1']
    df = df.drop('bar1', 1)
    df['bar1'] = [int(grade == 'P') for grade in df_bar]
    #df['race'] = [int(race == 7.0) for race in df['race']]
    #a = df['race']
    return df.to_numpy(), y.to_numpy()

def majority_pop(a):
    """
    Identify the main ethnicity group of each community
    """
    B = "racepctblack"
    W = "racePctWhite"
    A = "racePctAsian"
    H = "racePctHisp"
    races = [B, W, A, H]
    maj = a.apply(pd.Series.idxmax, axis=1)
    return maj

def clean_communities_full(path):
    """
    Extract black and white dominant communities; 
    sub_size : number of communities for each group
    """
    df = pd.read_csv(path)
    df = df.fillna(0)
    B = "racepctblack"
    W = "racePctWhite"
    A = "racePctAsian"
    H = "racePctHisp"
    sens_features = [2, 3, 4, 5]
    df_sens = df.iloc[:, sens_features]

    # creating labels using crime rate
    Y = df['ViolentCrimesPerPop']
    df = df.drop('ViolentCrimesPerPop', 1)

    maj = majority_pop(df_sens)

    # remap the values of maj
    a = maj.map({B : 0, W : 1, A : 2, H : 3})
   
    df['race'] = a
    df = df.drop(H, 1)
    df = df.drop(B, 1)
    df = df.drop(W, 1)
    df = df.drop(A, 1)

    #print(df.head())

    return df.to_numpy(), Y.to_numpy()

def house_price_load(trn_path,tst_path,save_data=False):

    train_csv = pd.read_csv(trn_path)
    test_csv = pd.read_csv(tst_path)

    drop_columns = (train_csv.isnull().sum().sort_values(ascending=False).\
        loc[lambda x : x> .90*1460]).index.to_list()

    train_clean = train_csv.drop(drop_columns, axis = 'columns', errors = 'ignore')
    test_clean = test_csv.drop(drop_columns, axis = 'columns', errors = 'ignore')

    train_10_percent_missing_features = train_clean.isnull().sum().sort_values(ascending=False).loc[lambda x : (x<.10*1460)  & (x != 0)].index.to_list()
    train_10_percent_missing_features_cat = train_clean[train_10_percent_missing_features].select_dtypes('object').columns.to_list()
    train_10_percent_missing_features_num = train_clean[train_10_percent_missing_features].select_dtypes('number').columns.to_list()

    train_clean[train_10_percent_missing_features_cat] = train_clean[train_10_percent_missing_features_cat].fillna(train_clean[train_10_percent_missing_features_cat].mode().iloc[0])
    train_clean[train_10_percent_missing_features_num] = train_clean[train_10_percent_missing_features_num].fillna(train_clean[train_10_percent_missing_features_num].median().iloc[0])

    test_10_percent_missing_features = test_clean.isnull().sum().sort_values(ascending=False).loc[lambda x : (x<.10*1460)  & (x != 0)].index.to_list()
    test_10_percent_missing_features_cat = test_clean[test_10_percent_missing_features].select_dtypes('object').columns.to_list()
    test_10_percent_missing_features_num = test_clean[test_10_percent_missing_features].select_dtypes('number').columns.to_list()

    test_clean[test_10_percent_missing_features_cat] = test_clean[test_10_percent_missing_features_cat].fillna(test_clean[test_10_percent_missing_features_cat].mode().iloc[0])
    test_clean[test_10_percent_missing_features_num] = test_clean[test_10_percent_missing_features_num].fillna(test_clean[test_10_percent_missing_features_num].median().iloc[0])

    train_clean["LotFrontage"] = train_clean["LotFrontage"].fillna(train_clean["LotFrontage"].median())
    test_clean["LotFrontage"] = test_clean["LotFrontage"].fillna(test_clean["LotFrontage"].median())

    train_clean.drop('Id', axis = 1, inplace = True)
    test_clean.drop('Id', axis = 1, inplace = True)

    trn_y = train_clean['SalePrice'].to_frame()
    #tst_y = test_clean['SalePrice'].to_frame()

    train_clean.drop(['SalePrice'], axis = 1, inplace = True)
    #test_clean.drop(['SalePrice'], axis = 1, inplace = True)
 
    X_train_clean_index = train_clean.index.to_list()
    X_total = train_clean.append(test_clean,ignore_index = True)
    X_test_clean_index = np.setdiff1d(X_total.index.to_list() ,X_train_clean_index)

    cat_features = X_total.select_dtypes(['object']).columns.to_list()
    X_total_encoded = MultiColumnLabelEncoder(columns = cat_features).fit_transform(X_total)
    
    X_train_clean_encoded = X_total_encoded.iloc[X_train_clean_index, :]
    X_test_clean_encoded = X_total_encoded.iloc[X_test_clean_index, :].reset_index(drop = True) 
    
    return X_train_clean_encoded.to_numpy(),trn_y.to_numpy()#,X_test_clean_encoded,tst_y

def community_crime_load(path,dim, save_data=False):

    data = []
    target = []
    with open(path) as fp:
        line = fp.readline()
        while line:
            temp = [i.strip() for i in line.strip().split(",")][5:]

            target.append(float(temp[-1]))
            
            temp_data = [0.0]*dim
            
            #print(temp)

            for i in range(len(temp[:-1])):

                if temp[i] != '?':
                    temp_data[i] = float(temp[i])
            
            data.append(temp_data)
            line = fp.readline()
    
    X_data = np.array(data, dtype=np.float32)
    Y_label = np.array(target)
    
    if save_data:
        # Save the numpy files to the folder where they come from
        data_np_path = path + '.data.npy'
        target_np_path = path + '.label.npy'
        np.save(data_np_path, X_data)
        np.save(target_np_path, Y_label)
    return (X_data, Y_label)

class CustomDataset(Dataset):
    def __init__(self, data, target, device=None, transform=None):       
        self.transform = transform
        if device is not None:
            # Push the entire data to given device, eg: cuda:0
            self.data = torch.from_numpy(data.astype('float32')).to(device)
            self.targets = torch.from_numpy(target).to(device)
        else:
            self.data = data#.astype('float32')
            self.targets = target

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()        
        sample_data = self.data[idx]
        label = self.targets[idx]
        if self.transform is not None:
            sample_data = self.transform(sample_data)
        return (sample_data, label) #.astype('float32')

def load_dataset_custom (datadir, dset_name,isnumpy=True):

    if dset_name == "Community_Crime":

        x_trn, y_trn = clean_communities_full(os.path.join(datadir, 'communities.csv'))

        if isnumpy:
            fullset = (x_trn, y_trn)
            
        else:
            fullset = CustomDataset(x_trn, y_trn)

        return fullset, x_trn.shape[1] 

    elif dset_name == 'LawSchool':

        x_trn, y_trn = clean_lawschool_full(os.path.join(datadir, 'lawschool.csv'))

        if isnumpy:
            fullset = (x_trn, y_trn)

        else:
            fullset = CustomDataset(x_trn, y_trn)
            
        return fullset, x_trn.shape[1] 

def load_std_regress_data (datadir, dset_name,isnumpy=True,clean=True):

    if dset_name == "cadata":
        trn_file = os.path.join(datadir, 'cadata.txt')
        x_trn, y_trn  = libsvm_file_load(trn_file,8)
    
    elif dset_name == "abalone":
        trn_file = os.path.join(datadir, 'abalone_scale.txt')
        x_trn, y_trn  = libsvm_file_load(trn_file,8)

    elif dset_name == "cpusmall":
        trn_file = os.path.join(datadir, 'cpusmall_scale.txt')
        x_trn, y_trn  = libsvm_file_load(trn_file,12)

    elif dset_name == "housing":
        trn_file = os.path.join(datadir, 'housing_scale.txt')
        x_trn, y_trn  = libsvm_file_load(trn_file,13)

    elif dset_name == "mg":
        trn_file = os.path.join(datadir, 'mg_scale.txt')
        x_trn, y_trn  = libsvm_file_load(trn_file,6)

    elif dset_name == "MSD":
        trn_file = os.path.join(datadir, 'YearPredictionMSD')
        x_trn, y_trn  = libsvm_file_load(trn_file,90)

    elif dset_name == "house_pricing":
        trn_file = os.path.join(datadir, 'train.csv')
        test_file = os.path.join(datadir, 'test.csv')
        x_trn, y_trn  = house_price_load(trn_file,test_file) #,x_tst,y_tst

    elif dset_name == "synthetic":

        data_dims = 30 #100
        samples = 5000#1000000

        np.random.seed(42)

        avg = np.zeros(data_dims)
        cov = np.identity(data_dims)

        x_trn = np.random.multivariate_normal(avg, cov, samples)#.T

        w = np.random.normal(0,10, data_dims) #0.25

        sigma = 30

        y_trn = x_trn.dot(w) + np.random.normal(0, sigma, samples)
    
    #elif dset_name == "house_pricing":
    #    x_trn, x_val, y_trn, y_val = train_test_split(x_trn, y_trn, test_size=0.1, random_state=42)

    
    if dset_name == "MSD":
        tst_file = os.path.join(datadir, 'YearPredictionMSD.t')
        x_tst, y_tst  = libsvm_file_load(tst_file,90)
        x_trn, x_val, y_trn, y_val = train_test_split(x_trn, y_trn, test_size=0.005, random_state=42)
    else:
        x_trn, x_tst, y_trn, y_tst = train_test_split(x_trn, y_trn, test_size=0.1, random_state=42)
        x_trn, x_val, y_trn, y_val = train_test_split(x_trn, y_trn, test_size=0.1, random_state=42)

    if not clean:

        noise_size = int(len(y_trn) * 0.5)
        noise_indices = np.random.choice(np.arange(len(y_trn)), size=noise_size, replace=False)
        
        sigma = 40
        y_trn[noise_indices] = y_trn[noise_indices] + np.random.normal(0, sigma, noise_size)

    sc = StandardScaler()
    x_trn = sc.fit_transform(x_trn)
    x_val = sc.transform(x_val)
    x_tst = sc.transform(x_tst)

    sc_l = StandardScaler()
    y_trn = np.reshape(sc_l.fit_transform(np.reshape(y_trn,(-1,1))),(-1))
    y_val = np.reshape(sc_l.fit_transform(np.reshape(y_val,(-1,1))),(-1))
    y_tst = np.reshape(sc_l.fit_transform(np.reshape(y_tst,(-1,1))),(-1))

    if isnumpy:
        fullset = (x_trn, y_trn)
        valset = (x_val, y_val)
        testset = (x_tst, y_tst)

    else:
        fullset = CustomDataset(x_trn, y_trn)
        valset = CustomDataset(x_val, y_val)
        testset = CustomDataset(x_tst, y_tst)

    return fullset, valset, testset





class Regression():
    def __init__(self):
        self.select_every = 35
        self.reg_lambda = 1e-5
        self.val_loss = 0
        self.test_loss = 0.
        self.test_loss_std = 0
        self.batch_size = 4000
        self.learning_rate = 0.01
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.psuedo_length = 1.0
        self.subset_idx = None
        
    def weight_reset(self, m):
        '''
        Fills the input tensor using Glorot Initialisation and

        '''
        torch.manual_seed(42)
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.1)
        
    def train_model(self, x_trn, y_trn, x_val, y_val, fraction, delt = [0.3], x_tst = None, y_tst = None,\
                    num_epochs=2000, default = False, ebud=None):
        
        sub_epoch = 3
        
        N, M = x_trn.shape
        bud = int(fraction * N)
        print("Budget, fraction and N:", bud, fraction, N)
        train_batch_size = min(bud,1000)
        print_every = 50

        deltas = torch.tensor(delt).to(self.device) 
        
      # initialise index
        rand_idxs = list(np.random.choice(N, size=bud, replace=False))
        idxs = rand_idxs

        criterion = nn.MSELoss()

        # Initialise model
        main_model = RegressionNet(M)
        main_model.apply(self.weight_reset)

        #for p in main_model.parameters():
        #    print(p.data)

        main_model = main_model.to(self.device)
        #criterion_sum = nn.MSELoss(reduction='sum')
        #main_optimizer = optim.SGD(main_model.parameters(), lr=learning_rate)
        main_optimizer = torch.optim.Adam(main_model.parameters(), lr = self.learning_rate)

        #scheduler = torch.optim.lr_scheduler.MultiStepLR(main_optimizer, milestones=change, gamma=0.5)
        scheduler = optim.lr_scheduler.StepLR(main_optimizer, step_size=1, gamma=0.1)

        #print(idxs)

        idxs.sort()
        np.random.seed(42)
        np_sub_idxs = np.array(idxs)
        np.random.shuffle(np_sub_idxs)
        loader_tr = DataLoader(CustomDataset(x_trn[np_sub_idxs], y_trn[np_sub_idxs],\
                transform=None),shuffle=False,batch_size=train_batch_size)

        loader_full_tr = DataLoader(CustomDataset(x_trn, y_trn,transform=None),shuffle=False,\
            batch_size=train_batch_size)

        loader_val = DataLoader(CustomDataset(x_val, y_val,transform=None),shuffle=False,\
            batch_size=self.batch_size)
        
        cached_state_dict = copy.deepcopy(main_model.state_dict())

        if self.psuedo_length == 1.0:
            sub_rand_idxs = [s for s in range(N)]
            current_idxs = idxs
        else:
            sub_rand_idxs = [s for s in range(N)]
            new_ele = set(sub_rand_idxs).difference(set(idxs))
            sub_rand_idxs = list(np.random.choice(list(new_ele), size=int(self.psuedo_length*N), replace=False))
            
            sub_rand_idxs = idxs + sub_rand_idxs

            current_idxs = [s for s in range(len(idxs))]

        fsubset_d = FindSubset_Vect_TrnLoss(x_trn[sub_rand_idxs], y_trn[sub_rand_idxs], x_val, y_val,main_model,\
            criterion,self.device,deltas,self.learning_rate,self.reg_lambda,self.batch_size)

        fsubset_d.precompute(int(num_epochs/4),sub_epoch,torch.randn_like(deltas,device=self.device))

        main_model.load_state_dict(cached_state_dict)
        
        print("Starting Subset of size ",fraction," with fairness Run!")
        
        stop_count = 0
        prev_loss = 1000
        prev_loss2 = 1000
        i =0
        mul=1
        lr_count = 0
        
        for i in range(num_epochs):
        
            temp_loss = 0.

            for batch_idx in list(loader_tr.batch_sampler):

                inputs, targets = loader_tr.dataset[batch_idx]
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                main_optimizer.zero_grad()

                scores = main_model(inputs)

                l2_reg = 0
                for param in main_model.parameters():
                    l2_reg += torch.norm(param)

                '''l = [torch.flatten(p) for p in main_model.parameters()]
                flat = torch.cat(l)
                l2_reg = torch.sum(flat*flat)'''

                loss = criterion(scores, targets) +  self.reg_lambda*l2_reg*len(batch_idx)
                temp_loss += loss.item()
                loss.backward()

                for p in filter(lambda p: p.grad is not None, main_model.parameters()):\
                     p.grad.data.clamp_(min=-.1, max=.1)

                main_optimizer.step()
                #scheduler.step()

            if i % print_every == 0:  # Print Training and Validation Loss
                print('Epoch:', i + 1, 'SubsetTrn', temp_loss)
                print("Previous loss: ", prev_loss, "\n"\
                    ,"Temporary loss: ", temp_loss, "\n"\
                    , "Mul: ", mul)
                # print(main_optimizer.param_groups[0]['lr'])
                
            if ((i + 1) % self.select_every == 0):
                
                cached_state_dict = copy.deepcopy(main_model.state_dict())
                clone_dict = copy.deepcopy(cached_state_dict)
                
                fsubset_d.lr = main_optimizer.param_groups[0]['lr']*mul#,1e-4)

                state_values = list(main_optimizer.state.values())
                step = state_values[0]['step']

                w_exp_avg = torch.cat((state_values[0]['exp_avg'].view(-1),state_values[1]['exp_avg']))
                #torch.zeros(x_trn.shape[1]+1,device=self.device)
                w_exp_avg_sq = torch.cat((state_values[0]['exp_avg_sq'].view(-1),state_values[1]['exp_avg_sq']))
                #torch.zeros(x_trn.shape[1]+1,device=self.device)

                #a_exp_avg = torch.zeros(1,device=self.device)
                #a_exp_avg_sq = torch.zeros(1,device=self.device)

                #print(exp_avg,exp_avg_sq)

                d_sub_idxs = fsubset_d.return_subset(clone_dict,sub_epoch,current_idxs,\
                    bud,train_batch_size,step,w_exp_avg,w_exp_avg_sq)
                #torch.ones_like(deltas,device=self.device),a_exp_avg,a_exp_avg_sq)#,main_optimizer,dual_optimizer)

                '''clone_dict = copy.deepcopy(cached_state_dict)
                alpha_orig = copy.deepcopy(alphas)

                sub_idxs = fsubset.return_subset(clone_dict,sub_epoch,sub_idxs,alpha_orig,bud,\
                    train_batch_size)
                print(sub_idxs[:10])'''

                current_idxs = d_sub_idxs
                #print(len(d_sub_idxs))

                d_sub_idxs = list(np.array(sub_rand_idxs)[d_sub_idxs])
                
                new_ele = set(d_sub_idxs).difference(set(idxs))
                # print(len(new_ele),0.1*bud)

                if len(new_ele) > 0.1*bud:
                    main_optimizer = torch.optim.Adam([
                    {'params': main_model.parameters()}], lr=main_optimizer.param_groups[0]['lr']*mul)
                    #max(main_optimizer.param_groups[0]['lr'],0.001))

                    #mul=1
                    #stop_count = 0
                    lr_count = 0

                idxs = d_sub_idxs

                idxs.sort()

                #print(idxs[:10])
                np.random.seed(42)
                np_sub_idxs = np.array(idxs)
                np.random.shuffle(np_sub_idxs)
                loader_tr = DataLoader(CustomDataset(x_trn[np_sub_idxs], y_trn[np_sub_idxs],\
                        transform=None),shuffle=False,batch_size=train_batch_size)

                main_model.load_state_dict(cached_state_dict)

            if abs(prev_loss - temp_loss) <= 1e-1*mul or prev_loss2 == temp_loss:
                #print(main_optimizer.param_groups[0]['lr'])
                lr_count += 1
                if lr_count == 10:
                    #print(i,"Reduced")
                    # print(prev_loss,temp_loss,main_optimizer.param_groups[0]['lr'])
                    scheduler.step()
                    mul/=10
                    lr_count = 0
            else:
                lr_count = 0

            prev_loss2 = prev_loss
            prev_loss = temp_loss
        
        no_red_error = torch.nn.MSELoss(reduction='none')

        main_model.eval()

        l = [torch.flatten(p) for p in main_model.parameters()]
        flat = torch.cat(l)

        print('Subset_fair',len(idxs))
        # print(flat)
        
        with torch.no_grad():

            #self.val_loss = 0.
            for batch_idx in list(loader_val.batch_sampler):

                inputs, targets = loader_val.dataset[batch_idx]
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                val_out = main_model(inputs)
                '''if is_time:
                    val_out = sc_trans.inverse_transform(val_out.cpu().numpy())
                    val_out = torch.from_numpy(val_out).float()'''

                if batch_idx[0] == 0:
                    e_val_loss = no_red_error(val_out, targets)

                else:
                    batch_val_loss = no_red_error(val_out, targets)
                    e_val_loss = torch.cat((e_val_loss, batch_val_loss),dim= 0)

            #val_loss /= len(loader_val.batch_sampler)
            self.val_loss = torch.mean(e_val_loss)
            # print(list(e_val_loss.cpu().numpy()))
            
            if (default == True):
                
                loader_tst = DataLoader(CustomDataset(x_tst, y_tst,transform=None),shuffle=False,\
                                        batch_size=self.batch_size)

                for batch_idx in list(loader_tst.batch_sampler):

                    inputs, targets = loader_tst.dataset[batch_idx]
                    inputs, targets = inputs.to(self.device), targets.to(self.device)

                    outputs = main_model(inputs)
                    '''if is_time:
                        outputs = sc_trans.inverse_transform(outputs.cpu().numpy())
                        outputs = torch.from_numpy(outputs).float()'''
                    #test_loss += criterion(outputs, targets)

                    if batch_idx[0] == 0:
                        e_tst_loss = no_red_error(outputs, targets)

                    else:
                        batch_tst_loss = no_red_error(outputs, targets)
                        e_tst_loss = torch.cat((e_tst_loss, batch_tst_loss),dim= 0)

                #test_loss /= len(loader_tst.batch_sampler)    
                self.test_loss = torch.mean(e_tst_loss)
                self.test_loss_std = torch.std(e_tst_loss)

                # print(list(e_tst_loss.cpu().numpy()))
                
    def train_model_fair(self, x_trn, y_trn, x_val, y_val, fraction,  delt = [0.3], x_tst = None, y_tst = None,\
                    num_epochs=2000, default = False, bud=None):

        sub_epoch = 3
        
        N, M = x_trn.shape
        bud = int(fraction * N)
        print("Budget, fraction and N:", bud, fraction, N)
        train_batch_size = min(bud,1000)
        print_every = 50

        deltas = torch.tensor(delt).to(self.device) 
        
      # initialise index
        rand_idxs = list(np.random.choice(N, size=bud, replace=False))
        sub_idxs = rand_idxs

        criterion = nn.MSELoss()

        main_model = RegressionNet(M)
        main_model.apply(self.weight_reset)

        #for p in main_model.parameters():
        #    print(p.data)

        main_model = main_model.to(self.device)

        #criterion_sum = nn.MSELoss(reduction='sum')

        alphas = torch.randn_like(deltas,device=self.device) #+ 5. #,requires_grad=True)
        alphas.requires_grad = True
        #print(alphas)
        #alphas = torch.ones_like(deltas,requires_grad=True)
        '''main_optimizer = optim.SGD([{'params': main_model.parameters()},
                    {'params': alphas}], lr=learning_rate) #'''
        main_optimizer = torch.optim.Adam(main_model.parameters(), lr=self.learning_rate)
        #[{'params': main_model.parameters()}], lr=learning_rate)

        dual_optimizer = torch.optim.Adam([{'params': alphas}], lr=self.learning_rate) #{'params': alphas} #'''

        #scheduler = torch.optim.lr_scheduler.MultiStepLR(main_optimizer, milestones=change,\
        #     gamma=0.5) #[e*2 for e in change]
        scheduler = optim.lr_scheduler.StepLR(main_optimizer, step_size=1, gamma=0.1)

        #alphas.requires_grad = False

        #delta_extend = torch.repeat_interleave(deltas,val_size, dim=0)    

        

        cached_state_dict = copy.deepcopy(main_model.state_dict())
        alpha_orig = copy.deepcopy(alphas)

        if self.psuedo_length == 1.0:
            sub_rand_idxs = [s for s in range(N)]
            current_idxs = sub_idxs
        else:
            sub_rand_idxs = [s for s in range(N)]
            new_ele = set(sub_rand_idxs).difference(set(sub_idxs))
            sub_rand_idxs = list(np.random.choice(list(new_ele), size=int(self.psuedo_length*N), replace=False))

            sub_rand_idxs = sub_idxs + sub_rand_idxs

            current_idxs = [s for s in range(len(sub_idxs))]

        fsubset_d = FindSubset_Vect(x_trn[sub_rand_idxs], y_trn[sub_rand_idxs], x_val, y_val,main_model,\
            criterion,self.device,deltas,self.learning_rate,self.reg_lambda,self.batch_size)

        fsubset_d.precompute(int(num_epochs/4),sub_epoch,alpha_orig)

        '''main_model.load_state_dict(cached_state_dict)
        alpha_orig = copy.deepcopy(alphas)

        fsubset = FindSubset(x_trn, y_trn, x_val, y_val,main_model,criterion,\
            self.device,deltas,learning_rate,reg_lambda)

        fsubset.precompute(int(num_epochs/4),sub_epoch,alpha_orig,batch_size)'''

        main_model.load_state_dict(cached_state_dict)

        print("Starting Subset of size ",fraction," with fairness Run!")

        sub_idxs.sort()
        np.random.seed(42)
        np_sub_idxs = np.array(sub_idxs)
        np.random.shuffle(np_sub_idxs)
        loader_tr = DataLoader(CustomDataset(x_trn[np_sub_idxs], y_trn[np_sub_idxs],\
                transform=None),shuffle=False,batch_size=train_batch_size)

        loader_val = DataLoader(CustomDataset(x_val, y_val,transform=None),shuffle=False,\
            batch_size=self.batch_size)

        loader_tst = DataLoader(CustomDataset(x_tst, y_tst,transform=None),shuffle=False,\
            batch_size=self.batch_size)

        stop_epoch = num_epochs

        #for i in range(num_epochs):
        stop_count = 0
        prev_loss = 1000
        prev_loss2 = 1000
        i =0
        mul = 1
        lr_count = 0
        #while (True):
        for i in range(num_epochs):

            # inputs, targets = x_trn[idxs].to(self.device), y_trn[idxs].to(self.device)
            #inputs, targets = x_trn[sub_idxs], y_trn[sub_idxs]

            temp_loss = 0.

            starting = time.process_time() 

            for batch_idx_t in list(loader_tr.batch_sampler):

                inputs_trn, targets_trn = loader_tr.dataset[batch_idx_t]
                inputs_trn, targets_trn = inputs_trn.to(self.device), targets_trn.to(self.device)

                main_optimizer.zero_grad()

                scores_trn = main_model(inputs_trn)

                l2_reg = 0
                for param in main_model.parameters():
                    l2_reg += torch.norm(param)

                '''l = [torch.flatten(p) for p in main_model.parameters()]
                flat = torch.cat(l)
                l2_reg = torch.sum(flat*flat)'''

                #state_orig = copy.deepcopy(main_optimizer.state)

                '''alpha_extend = torch.repeat_interleave(alphas,val_size, dim=0)
                val_scores = main_model(x_val_combined)
                constraint = criterion(val_scores, y_val_combined) - delta_extend
                multiplier = torch.dot(alpha_extend,constraint)'''

                '''constraint = torch.zeros(len(x_val_list))
                for j in range(len(x_val_list)):

                    inputs_j, targets_j = x_val_list[j], y_val_list[j]
                    scores_j = main_model(inputs_j)
                    constraint[j] = criterion(scores_j, targets_j) - deltas[j]'''

                constraint = 0.
                for batch_idx in list(loader_val.batch_sampler):

                    inputs, targets = loader_val.dataset[batch_idx]
                    inputs, targets = inputs.to(self.device), targets.to(self.device)

                    val_out = main_model(inputs)
                    '''if is_time:
                        val_out = sc_trans.inverse_transform(val_out.cpu().numpy())
                        val_out = torch.from_numpy(val_out).float()'''
                    constraint += criterion(val_out, targets)            

                constraint /= len(loader_val.batch_sampler)
                constraint = constraint - deltas
                multiplier = alphas*constraint*(float(constraint > 0)) #torch.dot(alphas,constraint)

                loss = criterion(scores_trn, targets_trn) + self.reg_lambda*l2_reg*len(batch_idx_t) + \
                    multiplier #
                temp_loss += loss.item()
                loss.backward()

                #if i % print_every == 0:  
                #    print(criterion(scores_trn, targets_trn) , reg_lambda*l2_reg*len(batch_idx_t) ,multiplier)

                # clamp gradients, just in case
                for p in filter(lambda p: p.grad is not None, main_model.parameters()):\
                     p.grad.data.clamp_(min=-.1, max=.1)

                main_optimizer.step()
                #scheduler.step()
                #main_optimizer.param_groups[1]['lr'] = learning_rate/2

                '''for param in main_model.parameters():
                    param.requires_grad = False
                alphas.requires_grad = True'''

                dual_optimizer.zero_grad()

                #if constraint > 0:
                constraint = 0.
                for batch_idx in list(loader_val.batch_sampler):

                    inputs, targets = loader_val.dataset[batch_idx]
                    inputs, targets = inputs.to(self.device), targets.to(self.device)

                    val_out = main_model(inputs)
                    '''if is_time:
                        val_out = sc_trans.inverse_transform(val_out.cpu().numpy())
                        val_out = torch.from_numpy(val_out).float()'''
                    constraint += criterion(val_out, targets)

                constraint /= len(loader_val.batch_sampler)
                constraint = constraint - deltas
                multiplier = -1.0*alphas*constraint*(float(constraint > 0)) #torch.dot(-1.0*alphas ,constraint)

                #print(alphas,constraint)

                #main_optimizer.state = state_orig
                multiplier.backward()
                dual_optimizer.step()
                #print(main_optimizer.param_groups)
                #scheduler.step()`

                alphas.requires_grad = False
                alphas.clamp_(min=0.0)
                alphas.requires_grad = True
                #print(alphas)

                '''for param in main_model.parameters():
                    param.requires_grad = True'''

            #print(alphas,constraint)

            if i % print_every == 0:  # Print Training and Validation Loss
                print('Epoch:', i + 1, 'SubsetTrn', loss.item())
                print("Previous loss: ", prev_loss, "\n"\
                    ,"Temporary loss: ", temp_loss, "\n"\
                    ,"Constraint: ", constraint, "\n"\
                    , "Alphas: ", alphas)
                #print(main_optimizer.state)#.keys())
                #print(alphas,constraint)
                #print(criterion(scores, targets) , reg_lambda*l2_reg*len(idxs) ,multiplier)
                #print(main_optimizer.param_groups)#[0]['lr'])


            if ((i + 1) % self.select_every == 0):

                cached_state_dict = copy.deepcopy(main_model.state_dict())
                clone_dict = copy.deepcopy(cached_state_dict)

                alpha_orig = alphas.detach().clone()#copy.deepcopy(alphas)

                '''alpha_orig.requires_grad = False
                alpha_orig = alpha_orig*((constraint >0).float())
                alpha_orig.requires_grad = True'''


                fsubset_d.lr = main_optimizer.param_groups[0]['lr']*mul#,1e-4)

                state_values = list(main_optimizer.state.values())
                step = 0#state_values[0]['step']

                w_exp_avg = torch.zeros(x_trn.shape[1]+1,device=self.device)
                #torch.cat((state_values[0]['exp_avg'].view(-1),state_values[1]['exp_avg']))
                w_exp_avg_sq = torch.zeros(x_trn.shape[1]+1,device=self.device)
                #torch.cat((state_values[0]['exp_avg_sq'].view(-1),state_values[1]['exp_avg_sq']))

                state_values = list(dual_optimizer.state.values())

                a_exp_avg = torch.zeros(1,device=self.device)
                #state_values[0]['exp_avg']
                a_exp_avg_sq = torch.zeros(1,device=self.device)
                #state_values[0]['exp_avg_sq']

                #print(exp_avg,exp_avg_sq)

                d_sub_idxs = fsubset_d.return_subset(clone_dict,sub_epoch,current_idxs,alpha_orig,bud,\
                    train_batch_size,step,w_exp_avg,w_exp_avg_sq,a_exp_avg,a_exp_avg_sq)#,main_optimizer,dual_optimizer)

                '''clone_dict = copy.deepcopy(cached_state_dict)
                alpha_orig = copy.deepcopy(alphas)

                sub_idxs = fsubset.return_subset(clone_dict,sub_epoch,sub_idxs,alpha_orig,bud,\
                    train_batch_size)
                print(sub_idxs[:10])'''

                current_idxs = d_sub_idxs

                d_sub_idxs = list(np.array(sub_rand_idxs)[d_sub_idxs])

                new_ele = set(d_sub_idxs).difference(set(sub_idxs))
                #print(len(new_ele),0.1*bud)

                if len(new_ele) > 0.1*bud:
                    main_optimizer = torch.optim.Adam([
                    {'params': main_model.parameters()}], lr=main_optimizer.param_groups[0]['lr'])
                    #max(main_optimizer.param_groups[0]['lr'],0.001))

                    dual_optimizer = torch.optim.Adam([{'params': alphas}], lr=self.learning_rate)

                    #mul=1
                    stop_count = 0
                    lr_count = 0

                sub_idxs = d_sub_idxs

                sub_idxs.sort()

                print("First 10 subset indices: ", sub_idxs[:10])
                np.random.seed(42)
                np_sub_idxs = np.array(sub_idxs)
                np.random.shuffle(np_sub_idxs)
                loader_tr = DataLoader(CustomDataset(x_trn[np_sub_idxs], y_trn[np_sub_idxs],\
                        transform=None),shuffle=False,batch_size=train_batch_size)

                main_model.load_state_dict(cached_state_dict)

            if abs(prev_loss - temp_loss) <= 1e-1*mul or abs(temp_loss - prev_loss2) <= 1e-1*mul:
                #print(main_optimizer.param_groups[0]['lr'])
                #print('lr',i)
                lr_count += 1
                if lr_count == 10:
                    # print(i,"Reduced",mul)
                    # print(prev_loss,temp_loss,alphas)
                    scheduler.step()
                    mul/=10
                    lr_count = 0
            else:
                lr_count = 0

            '''if (abs(prev_loss - temp_loss) <= 1e-3 or abs(temp_loss - prev_loss2) <= 1e-3) and\
                 stop_count >= 5:
                print(i,prev_loss,temp_loss,constraint)
                break 
            elif abs(prev_loss - temp_loss) <= 1e-3 or abs(temp_loss - prev_loss2) <= 1e-3:
                #print(prev_loss,temp_loss)
                stop_count += 1
            else:
                stop_count = 0'''

            '''if constraint <= 0 and (stop_count >= 2 or (i + 1) % select_every == 0): #10:
                print(i,constraint)
                break
            elif constraint <= 0:
                #print(alphas,constraint,stop_count)
                stop_count += 1
            else:
                stop_count = 0'''


            '''if i>=2000:
                break'''

            prev_loss2 = prev_loss
            prev_loss = temp_loss
            #i +=1

        self.subset_idx = sub_idxs
        #print(constraint)
        #print(alphas)
        no_red_error = torch.nn.MSELoss(reduction='none')

        #loader_tst = DataLoader(CustomDataset(x_tst, y_tst,transform=None),shuffle=False,\
        #    batch_size=batch_size)

        main_model.eval()

        l = [torch.flatten(p) for p in main_model.parameters()]
        flat = torch.cat(l)

        # print(func_name,len(sub_idxs),file=modelfile)
        # print(flat,file=modelfile)

        with torch.no_grad():
            '''full_trn_out = main_model(x_trn)
            full_trn_loss = criterion(full_trn_out, y_trn)
            sub_trn_out = main_model(x_trn[idxs])
            sub_trn_loss = criterion(sub_trn_out, y_trn[idxs])
            print("\nFinal SubsetTrn and FullTrn Loss:", sub_trn_loss.item(),full_trn_loss.item(),file=logfile)'''

            #val_loss = 0.
            for batch_idx in list(loader_val.batch_sampler):


                inputs, targets = loader_val.dataset[batch_idx]
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                val_out = main_model(inputs)
                '''if is_time:
                    val_out = sc_trans.inverse_transform(val_out.cpu().numpy())
                    val_out = torch.from_numpy(val_out).float()'''

                if batch_idx[0] == 0:
                    e_val_loss = no_red_error(val_out, targets)

                else:
                    batch_val_loss = no_red_error(val_out, targets)
                    e_val_loss = torch.cat((e_val_loss, batch_val_loss),dim= 0)

            #val_loss /= len(loader_val.batch_sampler)
            self.val_loss = torch.mean(e_val_loss)
            # print(list(e_val_loss.cpu().numpy()),file=modelfile)

            if(default == True):
                for batch_idx in list(loader_tst.batch_sampler):

                    inputs, targets = loader_tst.dataset[batch_idx]
                    inputs, targets = inputs.to(self.device), targets.to(self.device)

                    outputs = main_model(inputs)
                    '''if is_time:
                        outputs = sc_trans.inverse_transform(outputs.cpu().numpy())
                        outputs = torch.from_numpy(outputs).float()'''
                    #test_loss += criterion(outputs, targets)

                    if batch_idx[0] == 0:
                        e_tst_loss = no_red_error(outputs, targets)

                    else:
                        batch_tst_loss = no_red_error(outputs, targets)
                        e_tst_loss = torch.cat((e_tst_loss, batch_tst_loss),dim= 0)

                #test_loss /= len(loader_tst.batch_sampler)    
                self.test_loss = torch.mean(e_tst_loss)
                self.test_loss_std = torch.std(e_tst_loss)
                # print(list(e_tst_loss.cpu().numpy()),file=modelfile)    
       
    def val_loss(self):
        return self.val_loss.cpu().numpy()
    
    def test_loss(self):
        return (self.test_loss.cpu().numpy(), self.test_loss_std.cpu().numpy())

    def return_subset(self):
        return np.array(self.subset_idx)

In [3]:
reg = Regression()